In [38]:

import os

with open(".env", "w") as f:
    f.write("SECRET_KEY=egy_sk_test_736b619f9f89b6d8378e4973dcb4ea415884a8729ee4b0226700b5c3225f5613\n")
    f.write("PUBLIC_KEY=egy_pk_test_1Cob5RoKoxL9yDZcebyYZ3F59hiIVBGT\n")
    f.write("INTEGRATION_ID=5267090\n")
    f.write("REDIRECTION_URL=https://accept.paymob.com/unifiedcheckout/?publicKey=egy_pk_test_1Cob5RoKoxL9yDZcebyYZ3F59hiIVBGT&clientSecret;=egy_sk_test_736b619f9f89b6d8378e4973dcb4ea415884a8729ee4b0226700b5c3225f5613\n")
    f.write("NOTIFICATION_URL=https://accept.paymobsolutions.com/api/acceptance/post_pay\n")


In [42]:
import uuid
import json
import logging
import requests
from dotenv import load_dotenv

load_dotenv()

SECRET_KEY = os.getenv("SECRET_KEY")
PUBLIC_KEY = os.getenv("PUBLIC_KEY")
INTEGRATION_ID = os.getenv("INTEGRATION_ID")
REDIRECTION_URL = os.getenv("REDIRECTION_URL")
NOTIFICATION_URL = os.getenv("NOTIFICATION_URL")

if not SECRET_KEY or not PUBLIC_KEY or not INTEGRATION_ID:
    raise SystemExit("Missing required environment variables.")

INTENTION_URL = "https://accept.paymob.com/v1/intention/"


def create_intention(amount_egp: float, items=None, billing_data=None):
    merchant_order_id = f"order_{uuid.uuid4().hex[:12]}"
    amount_cents = int(round(amount_egp * 100))

    if billing_data is None:
        billing_data = {
            "apartment": "803",
            "floor": "8",
            "first_name": "Test",
            "last_name": "User",
            "street": "Test Street",
            "building": "10",
            "phone_number": "+201234567890",
            "shipping_method": "PKG",
            "city": "Cairo",
            "country": "EG",
            "state": "Cairo",
            "email": "test@example.com",
            "postal_code": "12345"
        }

    payload = {
        "amount": amount_cents,
        "currency": "EGP",
        "merchant_order_id": merchant_order_id,
        "integration_id": int(INTEGRATION_ID),
        "payment_methods": [int(INTEGRATION_ID)],
        "items": items or [],
        "billing_data": billing_data
    }

    if REDIRECTION_URL:
        payload["redirection_url"] = REDIRECTION_URL
    if NOTIFICATION_URL:
        payload["notification_url"] = NOTIFICATION_URL


    headers = {
        "Authorization": f"Token {SECRET_KEY}",
        "Content-Type": "application/json"
    }

    resp = requests.post(INTENTION_URL, headers=headers, data=json.dumps(payload), timeout=15)
    if resp.status_code not in [200, 201]:
        resp.raise_for_status()

    data = resp.json()
    return data

def build_unified_checkout_url(client_secret: str) -> str:
    return f"https://accept.paymob.com/unifiedcheckout/?publicKey={PUBLIC_KEY}&clientSecret={client_secret}"

if __name__ == "__main__":
    try:
        items = [
            {
                "name": "Test Product",
                "amount": 1000,
                "quantity": 1,
                "description": "Sample item",
                "image": None
            }
        ]

        resp = create_intention(amount_egp=10.00, items=items)
        client_secret = resp.get("client_secret")
        checkout_url = build_unified_checkout_url(client_secret)
        print("\n Open this URL to complete payment:")
        print(checkout_url)

    except Exception as e:
        print("Failed to create intention: %s", e)



 Open this URL to complete payment:
https://accept.paymob.com/unifiedcheckout/?publicKey=egy_pk_test_1Cob5RoKoxL9yDZcebyYZ3F59hiIVBGT&clientSecret=egy_csk_test_67558216308b819beb20c71d6f513bfb
